In [1]:
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551
G = (Gx, Gy)
INF = (0, 1, 0)

def mod_pow(x, y, m):
    """计算 (x^y) % m"""
    return pow(x, y, m)

def extended_gcd(a, b):
    """扩展欧几里得算法"""
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = extended_gcd(b % a, a)
        return (g, y - (b // a) * x, x)

def mod_inverse(a, m):
    """计算模反元素，即找到b使得(a*b) % m == 1"""
    g, x, y = extended_gcd(a, m)
    if g != 1 and a % m == 0:
        return 0 
    else:
        return x % m

In [2]:
def jacobian_point_add(point_p, point_q):
    """
    在雅可比坐标系下执行两个椭圆曲线点的加法运算 (point_p + point_q)
    
    参数:
    point_p: 三元组 (X1, Y1, Z1) 表示雅可比坐标系中的点
    point_q: 三元组 (X2, Y2, Z2) 表示雅可比坐标系中的点
    
    返回:
    雅可比坐标系中的点加法结果，或无穷远点 INF
    """
    # 处理无穷远点的特殊情况
    if point_p == INF:
        return point_q
    if point_q == INF:
        return point_p
    
    # 提取坐标分量
    x1, y1, z1 = point_p
    x2, y2, z2 = point_q
    
    # 检查点是否互为相反数
    if (x1 * mod_pow(z2, 2, p) - x2 * mod_pow(z1, 2, p)) % p == 0:
        if (y1 * mod_pow(z2, 3, p) + y2 * mod_pow(z1, 3, p)) % p == 0:
            return INF  # 结果为无穷远点
    
    # 计算中间值
    z1_squared = (z1 * z1) % p
    z2_squared = (z2 * z2) % p
    u1 = (x1 * z2_squared) % p
    u2 = (x2 * z1_squared) % p
    s1 = (y1 * z2 * z2_squared) % p
    s2 = (y2 * z1 * z1_squared) % p
    
    # 检查点是否相同（可能需要倍乘）
    if u1 == u2:
        if s1 != s2:
            return INF  # 结果为无穷远点
        return jacobian_point_double(point_p)  # 执行点倍乘
    
    # 计算点加法
    h = (u2 - u1) % p
    r = (s2 - s1) % p
    h_squared = (h * h) % p
    h_cubed = (h * h_squared) % p
    v = (u1 * h_squared) % p
    x3 = (r * r - h_cubed - 2 * v) % p
    y3 = (r * (v - x3) - s1 * h_cubed) % p
    z3 = (z1 * z2 * h) % p
    
    return (x3, y3, z3)

def jacobian_point_double(point_p):
    """
    在雅可比坐标系下执行椭圆曲线点的倍乘运算 (2 * point_p)
    
    参数:
    point_p: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    
    返回:
    雅可比坐标系中的点倍乘结果，或无穷远点 INF
    """
    # 处理无穷远点和Y坐标为0的特殊情况
    if point_p == INF:
        return INF
    x, y, z = point_p
    if y == 0:
        return INF  # Y坐标为0表示无穷远点
    
    # 计算中间值
    y_squared = (y * y) % p
    y_fourth = (y_squared * y_squared) % p
    s = (4 * x * y_squared) % p
    m = (3 * x * x + a * z * z * z * z) % p
    
    # 计算倍乘结果
    x3 = (m * m - 2 * s) % p
    y3 = (m * (s - x3) - 8 * y_fourth) % p
    z3 = (2 * y * z) % p
    
    return (x3, y3, z3)

def jacobian_point_multiply(point_p, scalar_k):
    """
    在雅可比坐标系下执行椭圆曲线点的标量乘法运算 (scalar_k * point_p)
    
    参数:
    point_p: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    scalar_k: 整数，表示标量值
    
    返回:
    雅可比坐标系中的标量乘法结果，或无穷远点 INF
    """
    # 处理标量为0的特殊情况
    if scalar_k == 0:
        return INF
    
    # 使用二进制分解法执行标量乘法
    result = INF
    current_addend = point_p
    
    while scalar_k > 0:
        # 如果当前位为1，则累加当前加数
        if scalar_k & 1:
            result = jacobian_point_add(result, current_addend)
        
        # 加倍当前加数并右移标量
        current_addend = jacobian_point_double(current_addend)
        scalar_k >>= 1
    
    return result

def convert_jacobian_to_affine(jacobian_point):
    """
    将雅可比坐标系中的点转换为仿射坐标系中的点
    
    参数:
    jacobian_point: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    
    返回:
    二元组 (x, y) 表示仿射坐标系中的点，或None表示无穷远点
    """
    # 处理无穷远点
    if jacobian_point == INF:
        return None  # 无穷远点在仿射坐标系中表示为None
    
    # 提取坐标分量
    x_jacobian, y_jacobian, z_jacobian = jacobian_point
    
    # 计算Z的逆元及其幂
    z_inverse = mod_inverse(z_jacobian, p)
    z_squared_inverse = (z_inverse * z_inverse) % p
    z_cubed_inverse = (z_squared_inverse * z_inverse) % p
    
    # 转换为仿射坐标
    x_affine = (x_jacobian * z_squared_inverse) % p
    y_affine = (y_jacobian * z_cubed_inverse) % p
    
    return (x_affine, y_affine)

In [3]:
def montgomery_init(modulus):
    """
    初始化蒙哥马利模运算所需的参数
    
    参数:
    modulus: 模运算的模数，必须为正整数
    
    返回:
    包含蒙哥马利参数的元组 (modulus, radix, radix_inverse, modulus_inverse)
    """
    radix = 1 << modulus.bit_length()
    radix_inverse = mod_inverse(radix, modulus)
    _, inv, _ = extended_gcd(modulus, radix)
    modulus_inverse = (radix - (inv % radix)) % radix
    return modulus, radix, radix_inverse, modulus_inverse

def convert_to_montgomery(x, params):
    """
    将普通整数转换为蒙哥马利形式
    
    参数:
    x: 要转换的普通整数
    params: 蒙哥马利参数元组 (modulus, radix, radix_inverse, modulus_inverse)
    
    返回:
    x的蒙哥马利形式表示
    """
    modulus, radix, _, _ = params
    return (x * radix) % modulus

def convert_from_montgomery(x, params):
    """
    将蒙哥马利形式的数转换为普通整数
    
    参数:
    x: 要转换的蒙哥马利形式的数
    params: 蒙哥马利参数元组 (modulus, radix, radix_inverse, modulus_inverse)
    
    返回:
    x的普通整数表示
    """
    modulus, _, radix_inverse, _ = params
    return (x * radix_inverse) % modulus

def montgomery_mul(a, b, params):
    """
    执行蒙哥马利模乘运算 [a * b * (R^-1)] mod modulus
    
    参数:
    a, b: 蒙哥马利形式的操作数
    params: 蒙哥马利参数元组 (modulus, radix, radix_inverse, modulus_inverse)
    
    返回:
    蒙哥马利形式的乘法结果
    """
    modulus, radix, _, modulus_inverse = params
    bit_length = modulus.bit_length()
    
    multiply = a * b
    temp = (multiply * modulus_inverse) & (radix - 1)
    res = (multiply + temp * modulus) >> bit_length
    return res % modulus

def montgomery_add(a, b, params):
    """
    执行蒙哥马利域中的加法运算 a + b mod modulus
    
    参数:
    a, b: 蒙哥马利形式的操作数
    params: 蒙哥马利参数元组 (modulus, radix, radix_inverse, modulus_inverse)
    
    返回:
    蒙哥马利形式的加法结果
    """
    modulus, _, _, _ = params
    return (a + b) % modulus

def montgomery_sub(a, b, params):
    """
    执行蒙哥马利域中的减法运算 a - b mod modulus
    
    参数:
    a, b: 蒙哥马利形式的操作数
    params: 蒙哥马利参数元组 (modulus, radix, radix_inverse, modulus_inverse)
    
    返回:
    蒙哥马利形式的减法结果
    """
    modulus, _, _, _ = params
    return (a - b) % modulus

def montgomery_jacobian_point_add(point_p, point_q, curve_a_montgomery, montgomery_params):
    """
    在雅可比坐标系下使用蒙哥马利模乘执行椭圆曲线点加法
    
    参数:
    point_p: 三元组 (X1, Y1, Z1) 表示雅可比坐标系中的点
    point_q: 三元组 (X2, Y2, Z2) 表示雅可比坐标系中的点
    curve_a_montgomery: 曲线参数a的蒙哥马利形式
    montgomery_params: 蒙哥马利模运算参数元组
    
    返回:
    点加法结果的雅可比坐标，或无穷远点INF
    """
    if point_p == INF:
        return point_q
    if point_q == INF:
        return point_p
    
    x1, y1, z1 = point_p
    x2, y2, z2 = point_q
    
    # 计算中间值
    z1_squared = montgomery_mul(z1, z1, montgomery_params)
    z2_squared = montgomery_mul(z2, z2, montgomery_params)
    u1 = montgomery_mul(x1, z2_squared, montgomery_params)
    u2 = montgomery_mul(x2, z1_squared, montgomery_params)
    
    z2_cubed = montgomery_mul(z2, z2_squared, montgomery_params)
    z1_cubed = montgomery_mul(z1, z1_squared, montgomery_params)
    s1 = montgomery_mul(y1, z2_cubed, montgomery_params)
    s2 = montgomery_mul(y2, z1_cubed, montgomery_params)
    
    # 检查特殊情况
    if u1 == u2:
        if s1 == montgomery_sub(0, s2, montgomery_params):
            return INF  # 点互为相反数，结果为无穷远点
            
    if u1 == u2 and s1 == s2:
        return montgomery_jacobian_point_double(point_p, curve_a_montgomery, montgomery_params)
    
    # 执行点加法计算
    h = montgomery_sub(u2, u1, montgomery_params)
    r = montgomery_sub(s2, s1, montgomery_params)
    h_squared = montgomery_mul(h, h, montgomery_params)
    h_cubed = montgomery_mul(h, h_squared, montgomery_params)
    v = montgomery_mul(u1, h_squared, montgomery_params)
    
    two_montgomery = convert_to_montgomery(2, montgomery_params)
    x3 = montgomery_sub(
        montgomery_sub(montgomery_mul(r, r, montgomery_params), h_cubed, montgomery_params),
        montgomery_mul(v, two_montgomery, montgomery_params),
        montgomery_params
    )
    
    y3 = montgomery_sub(
        montgomery_mul(r, montgomery_sub(v, x3, montgomery_params), montgomery_params),
        montgomery_mul(s1, h_cubed, montgomery_params),
        montgomery_params
    )
    
    z3 = montgomery_mul(
        montgomery_mul(z1, z2, montgomery_params),
        h,
        montgomery_params
    )
    
    return (x3, y3, z3)

def montgomery_jacobian_point_double(point_p, curve_a_montgomery, montgomery_params):
    """
    在雅可比坐标系下使用蒙哥马利模乘执行椭圆曲线点倍乘
    
    参数:
    point_p: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    curve_a_montgomery: 曲线参数a的蒙哥马利形式
    montgomery_params: 蒙哥马利模运算参数元组
    
    返回:
    点倍乘结果的雅可比坐标，或无穷远点INF
    """
    if point_p == INF:
        return INF
    
    x, y, z = point_p

    if y == 0:
        return INF  # Y坐标为0表示无穷远点
    
    # 计算中间值
    y_squared = montgomery_mul(y, y, montgomery_params)
    y_fourth = montgomery_mul(y_squared, y_squared, montgomery_params)
    
    four_montgomery = convert_to_montgomery(4, montgomery_params)
    s = montgomery_mul(
        four_montgomery,
        montgomery_mul(x, y_squared, montgomery_params),
        montgomery_params
    )
    
    three_montgomery = convert_to_montgomery(3, montgomery_params)
    x_squared = montgomery_mul(x, x, montgomery_params)
    three_x_squared = montgomery_mul(three_montgomery, x_squared, montgomery_params)
    
    z_squared = montgomery_mul(z, z, montgomery_params)
    z_fourth = montgomery_mul(z_squared, z_squared, montgomery_params)
    a_z_fourth = montgomery_mul(curve_a_montgomery, z_fourth, montgomery_params)
    
    # 计算倍乘结果
    m = montgomery_add(three_x_squared, a_z_fourth, montgomery_params)
    
    two_montgomery = convert_to_montgomery(2, montgomery_params)
    x3 = montgomery_sub(
        montgomery_mul(m, m, montgomery_params),
        montgomery_mul(s, two_montgomery, montgomery_params),
        montgomery_params
    )
    
    eight_montgomery = convert_to_montgomery(8, montgomery_params)
    y3 = montgomery_sub(
        montgomery_mul(m, montgomery_sub(s, x3, montgomery_params), montgomery_params),
        montgomery_mul(eight_montgomery, y_fourth, montgomery_params),
        montgomery_params
    )
    
    z3 = montgomery_mul(
        two_montgomery,
        montgomery_mul(y, z, montgomery_params),
        montgomery_params
    )
    
    return (x3, y3, z3)

def montgomery_jacobian_point_multiply(scalar_k, point_p, curve_a_montgomery, montgomery_params):
    """
    在雅可比坐标系下使用蒙哥马利模乘执行椭圆曲线点标量乘法
    
    参数:
    scalar_k: 整数，表示标量值
    point_p: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    curve_a_montgomery: 曲线参数a的蒙哥马利形式
    montgomery_params: 蒙哥马利模运算参数元组
    
    返回:
    标量乘法结果的雅可比坐标，或无穷远点INF
    """
    if scalar_k == 0:
        return INF
        
    result = INF
    current_point = point_p
    
    # 二进制分解法执行标量乘法
    while scalar_k > 0:
        if scalar_k & 1:
            result = montgomery_jacobian_point_add(
                result, current_point, curve_a_montgomery, montgomery_params
            )
        current_point = montgomery_jacobian_point_double(
            current_point, curve_a_montgomery, montgomery_params
        )
        scalar_k >>= 1
        
    return result

def convert_montgomery_jacobian_to_affine(jacobian_point, montgomery_params):
    """
    将蒙哥马利雅可比坐标转换为普通仿射坐标
    
    参数:
    jacobian_point: 三元组 (X, Y, Z) 表示雅可比坐标系中的点
    montgomery_params: 蒙哥马利模运算参数元组
    
    返回:
    二元组 (x, y) 表示仿射坐标系中的点，或None表示无穷远点
    """
    if jacobian_point == INF:
        return None
    
    x_jacobian, y_jacobian, z_jacobian = jacobian_point
    modulus, _, _, _ = montgomery_params
    
    # 转换为普通数
    z_normal = convert_from_montgomery(z_jacobian, montgomery_params)
    
    # 计算逆元
    z_inv = mod_inverse(z_normal, modulus)
    z_squared_inv = (z_inv * z_inv) % modulus
    z_cubed_inv = (z_squared_inv * z_inv) % modulus
    
    # 转换为仿射坐标
    x_affine = (convert_from_montgomery(x_jacobian, montgomery_params) * z_squared_inv) % modulus
    y_affine = (convert_from_montgomery(y_jacobian, montgomery_params) * z_cubed_inv) % modulus
    
    return (x_affine, y_affine)

In [5]:
G_jacobian = (Gx, Gy, 1)

# 计算2G
res_2G_jacobian = jacobian_point_double(G_jacobian)
res_2G_affine = convert_jacobian_to_affine(res_2G_jacobian)

# 计算3G (2G + G)
res_3G_jacobian = jacobian_point_add(res_2G_jacobian, G_jacobian)
res_3G_affine = convert_jacobian_to_affine(res_3G_jacobian)

# 计算31G (使用标量乘法)
res_31G_jacobian = jacobian_point_multiply(G_jacobian, 31)
res_31G_affine = convert_jacobian_to_affine(res_31G_jacobian)

# 辅助函数：将点坐标转换为十六进制字符串
def to_hex(point):
    if point is None:
        return "None"
    if isinstance(point, tuple):
        return tuple(f"0x{x:064x}" for x in point)
    return f"0x{point:064x}"

# 打印十六进制结果
print("2G的雅可比坐标:", to_hex(res_2G_jacobian))
print("2G的仿射坐标:", to_hex(res_2G_affine))
print("\n3G的雅可比坐标:", to_hex(res_3G_jacobian))
print("3G的仿射坐标:", to_hex(res_3G_affine))
print("\n31G的雅可比坐标:", to_hex(res_31G_jacobian))
print("31G的仿射坐标:", to_hex(res_31G_affine))

2G的雅可比坐标: ('0x9a978f59acd1b5ad570e7d52dcfcde43804b42274f61ddcf1e7d848391d6c70f', '0x4126885e7f786af905338238e5346d5fe77fc46388668bd0fd59be3190d2f5d1', '0x9fc685c5fc34ff371dcfd694f81f3c2c579c66aed662bd9d976c80d06f7ea3ea')
2G的仿射坐标: ('0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978', '0x07775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1')

3G的雅可比坐标: ('0x0ec88440c8b00a9e572bf1bceb7d0c5906bd65990a9b7081130bd72e2c136ca0', '0xc0bc77e1339e899101f8e8eccf79c3f7f4bbdd1bf96f6446199bd423026a60d6', '0xdd27cb52a31d1f6e041accf1103de05ba0a5edd74b738d51fe3397de0e3fc306')
3G的仿射坐标: ('0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c', '0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb83d9a79b127a27d5032')

31G的雅可比坐标: ('0x2308c580be979559af21da942a396e634676dc1ee604f27029bb20479fdd496b', '0xde157146d19f8efbd8eb9c5b3ccc8646e62e7c490a1110a1d02127900854721f', '0xc39f3be4cb3e64a988342896fcc835da0b9593d6158ecc8fecfaa7fdc9e2c5f0')
31G的仿射坐标: ('0x301d9e502dc7e05da85d

In [6]:
p = 7
INF = (0, 1, 0)

def mod_pow(x, y, m):
    """计算 (x^y) % m"""
    return pow(x, y, m)

def extended_gcd(a, b):
    """扩展欧几里得算法"""
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = extended_gcd(b % a, a)
        return (g, y - (b // a) * x, x)

def mod_inverse(a, m):
    """计算模反元素，即找到b使得(a*b) % m == 1"""
    g, x, y = extended_gcd(a, m)
    if g != 1:
        if a % m == 0:
            return 0 
        else:
            raise Exception(f'模反元素不存在: gcd({a}, {m}) = {g}')
    else:
        return x % m

def point_double_jacobian(P):
    """雅可比坐标系下的点倍乘 2P"""
    if P == INF:
        return INF
    X, Y, Z = P
    if Y == 0:
        return INF
    Y2 = (Y * Y) % p
    Y4 = (Y2 * Y2) % p
    S = (4 * X * Y2) % p
    M = (3 * X * X + a * Z * Z * Z * Z) % p
    X3 = (M * M - 2 * S) % p
    Y3 = (M * (S - X3) - 8 * Y4) % p
    Z3 = (2 * Y * Z) % p
    return (X3, Y3, Z3)

def point_add_jacobian(P, Q):
    """雅可比坐标系下的点加法 P + Q"""
    if P == INF:
        return Q
    if Q == INF:
        return P
    X1, Y1, Z1 = P
    X2, Y2, Z2 = Q
    if (X1 * mod_pow(Z2, 2, p) - X2 * mod_pow(Z1, 2, p)) % p == 0:
        if (Y1 * mod_pow(Z2, 3, p) + Y2 * mod_pow(Z1, 3, p)) % p == 0:
            return INF
    Z1Z1 = (Z1 * Z1) % p
    Z2Z2 = (Z2 * Z2) % p
    U1 = (X1 * Z2Z2) % p
    U2 = (X2 * Z1Z1) % p
    S1 = (Y1 * Z2 * Z2Z2) % p
    S2 = (Y2 * Z1 * Z1Z1) % p
    if U1 == U2:
        if S1 != S2:
            return INF
        return point_double_jacobian(P)  
    H = (U2 - U1) % p
    R = (S2 - S1) % p
    H2 = (H * H) % p
    H3 = (H * H2) % p
    V = (U1 * H2) % p
    X3 = (R * R - H3 - 2 * V) % p
    Y3 = (R * (V - X3) - S1 * H3) % p
    Z3 = (Z1 * Z2 * H) % p
    return (X3, Y3, Z3)

def jacobian_to_affine(P):
    """将雅可比坐标转换为仿射坐标"""
    if P == INF:
        return None  
    X, Y, Z = P
    Z_inv = mod_inverse(Z, p)
    Z2_inv = (Z_inv * Z_inv) % p
    Z3_inv = (Z2_inv * Z_inv) % p
    x = (X * Z2_inv) % p
    y = (Y * Z3_inv) % p
    return (x, y)

res = point_add_jacobian((1,0,1), (4,3,1))
print(res)
print(jacobian_to_affine(res))

(6, 2, 3)
(3, 5)
